# BTRC: A simple tool to crop 3D images

The scans has too many empty spaces, so ```cropped_images``` is simple function to crop all superfluous area (pixels with 0 value near the edge) in 3D images

In [ ]:
import numpy as np 
from glob import glob
import plotly.express as px
import pydicom

def read_dicom_files(cohort, case, mpMRI):
    PATH = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
    files_glob = f'{PATH}/{cohort}/{case}/{mpMRI}/*.dcm'
    sorted_files = sorted(glob(files_glob),key=lambda f: int(f.split('Image-')[1].split('.')[0]))
    return [pydicom.read_file(f) for f in sorted_files]

In [ ]:
def cropped_images(images):
    min=np.array(np.nonzero(images)).min(axis=1)
    max=np.array(np.nonzero(images)).max(axis=1)+1 # --> Thank you @lai321!
    return images[min[0]:max[0],min[1]:max[1],min[2]:max[2]]

### Test

In [ ]:
cuore = np.array(1).reshape(1,1,1)
padded_array = np.pad(cuore,1)
assert cropped_images(padded_array) == cuore

### Example

In [ ]:
cohort = 'train'
case = '00000'
mpMRI = 'FLAIR'
dicom_files = read_dicom_files(cohort, case, mpMRI)

images = np.array([s.pixel_array for s in dicom_files])
images = cropped_images(images)

fig = px.imshow(images, animation_frame=0, binary_string=True, labels=dict(animation_frame="scan"), height=600)
fig.show()